In [5]:
import pandas as pd
import re
import numpy as np
water_quality_df = pd.read_csv('Overall-dataset-llda(not_cleaned).csv')
weather_df = pd.read_csv('New_Weather.csv')

water_quality_df['Phytoplankton (cells/ml)'] = water_quality_df['Phytoplankton (cells/ml)'].str.replace(',', '')

water_quality_df['Month'] = water_quality_df['Month'].astype(str)
water_quality_df.replace('-', np.nan, inplace=True)
# Correct typos in the 'Month' column
water_quality_df['Month'] = water_quality_df['Month'].replace({
    'Febuary': 'February',
    'Aug': 'August',
    'Sept': 'September',
    'Nov': 'November',
    'Dec': 'December'
}, regex=False)  # Use regex=False to avoid treating the keys as regular expressions

# Display the updated DataFrame to verify the changes

# List of columns to exclude from transformation
exclude_columns = ['Month', 'Wind', 'Condition']

# Function to remove non-numeric characters from each cell
def remove_non_numeric(value):
    if isinstance(value, str) and value not in exclude_columns:
        return re.sub(r'[^0-9.]', '', value)
    else:
        return value

# Apply the function to each cell in numeric columns of the dataframe
for column in weather_df.columns:
    if column not in exclude_columns:
        weather_df[column] = weather_df[column].apply(remove_non_numeric)

# Convert the numeric columns to numeric type
numeric_columns = [col for col in weather_df.columns if col not in exclude_columns]
weather_df[numeric_columns] = weather_df[numeric_columns].apply(pd.to_numeric, errors='coerce')

# Define mappings for Wind column
wind_mapping = {
    'N': 1, 'NNE': 2, 'NE': 3, 'ENE': 4,
    'E': 5, 'ESE': 6, 'SE': 7, 'SSE': 8,
    'S': 9, 'SSW': 10, 'SW': 11, 'WSW': 12,
    'W': 13, 'WNW': 14, 'NW': 15, 'NNW': 16,
    'VAR': 17, 'CALM': 18
}

# Define mappings for Condition column
condition_mapping = {
    'Fair': 1, 'Mostly Cloudy': 2, 'Partly Cloudy': 3, 'Cloudy': 4,
    'Light Rain': 5, 'Light Rain Shower': 6, 'Rain': 7, 'Heavy Rain': 8,
    'Thunder': 9, 'Light Rain with Thunder': 10, 'T-Storm': 11,
    'Heavy Rain Shower': 12, 'Rain Shower': 13, 'Showers in the Vicinity': 14,
    'Thunder in the Vicinity': 15, 'Mostly Cloudy / Windy': 16,
    'Fair / Windy': 17, 'Partly Cloudy / Windy': 18, 'Rain / Windy': 19,
    'Light Rain Shower / Windy': 20, 'Heavy Rain / Windy': 21
}

# Apply mappings to Wind and Condition columns
weather_df['Wind'] = weather_df['Wind'].map(wind_mapping)
weather_df['Condition'] = weather_df['Condition'].map(condition_mapping)



# Define a function to compute the mode
def compute_mode(series):
    return series.mode().iloc[0] if not series.mode().empty else None

# Group by 'Year' and 'Month'
grouped = weather_df.groupby(['Year', 'Month'])

# Aggregate to get the mean for all columns except 'Wind' and 'Condition'
# and the mode for 'Wind' and 'Condition'
weather_monthly_stats = grouped.agg({
    'Wind': compute_mode,
    'Condition': compute_mode,
    'Day': compute_mode,
    'Time': 'mean',
    'Temperature': 'mean',
    'Dew Point' : 'mean',
    'Humidity': 'mean',
    'Wind Speed': 'mean',
    'Wind Gust': 'mean',
    'Pressure': 'mean',
    'Precip.': 'mean'
}).reset_index()

# Print the result
print(weather_monthly_stats)


# Merge datasets on 'Month' and 'Year'
merged_df = pd.merge(water_quality_df, weather_monthly_stats, on=['Month', 'Year'])

print(merged_df['Month'].unique())

# Step 8: Save the updated dataset
output_file_path = '2023_Merged.csv'
merged_df.to_csv(output_file_path, index=False)


     Year      Month  Wind  Condition  Day        Time  Temperature  \
0    2013      April   6.0        1.0    1  650.000000    30.375000   
1    2013     August   5.0        2.0    5  650.000000    29.208333   
2    2013   December  18.0        2.0    2  650.000000    26.625000   
3    2013   February   6.0        1.0    4  652.173913    27.434783   
4    2013    January   1.0        2.0   14  650.000000    25.875000   
..    ...        ...   ...        ...  ...         ...          ...   
124  2023      March  17.0        1.0    6  650.000000    26.875000   
125  2023        May   6.0        1.0    1  650.000000    31.166667   
126  2023   November   5.0        1.0    6  632.600000    30.280000   
127  2023    October  12.0        2.0    2  636.185185    28.444444   
128  2023  September  12.0        5.0    4  649.440000    27.320000   

     Dew Point   Humidity  Wind Speed  Wind Gust     Pressure  Precip.  
0    22.208333  62.041667   12.750000   0.000000  1009.076250      0.0  
1

In [3]:
import pandas as pd

# Read the CSV file with ISO-8859-1 encoding
df = pd.read_csv('Overall-weather-dataset(csv_converted_2023).csv', encoding='iso-8859-1')

# Write the CSV file with UTF-8 encoding
df.to_csv('New_Weather.csv', encoding='utf-8', index=False)


In [16]:
import pandas as pd

# Load the dataset
df2 = pd.read_csv('2023_Merged.csv')

# Map month names to numbers
month_mapping = {
    'January': 1,
    'February': 2,
    'March': 3,
    'April': 4,
    'May': 5,
    'June': 6,
    'July': 7,
    'August': 8,
    'September': 9,
    'October': 10,
    'November': 11,
    'December': 12
}
df2['Month'] = df2['Month'].map(month_mapping)

# Convert 'Year', 'Month', 'Day' into a single 'Date' column
df2['Date'] = pd.to_datetime(df2[['Year', 'Month', 'Day']])

# Optionally, set 'Date' as the index if needed
# df2.set_index('Date', inplace=True)

# Save the DataFrame to a new CSV file
df2.to_csv('Complete.csv', encoding='utf-8', index=False)


Filling missing values with same month and weather

In [9]:
import pandas as pd
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import euclidean

# Load the dataset
df = pd.read_csv('New_Merged.csv')

# Columns of interest
weather_cols = ['Temperature', 'Dew Point', 'Humidity', 'Wind', 'Wind Speed', 
                'Wind Gust', 'Pressure', 'Precip.', 'Condition']
water_quality_cols = ['pH (units)', 'Ammonia (mg/L)', 'Nitrate (mg/L)', 
                      'Inorganic Phosphate (mg/L)', 'BOD (mg/l)', 
                      'Dissolved Oxygen (mg/l)', 'Total coliforms (MPN/100ml)']

# Update the code to include "Phytoplankton (cells/ml)" in the comparison
def phytoplankton_distance(row1, row2, threshold=0.1):
    """Calculate the distance between phytoplankton values, considering them close if within the threshold."""
    return abs(row1['Phytoplankton (cells/ml)'] - row2['Phytoplankton (cells/ml)']) <= threshold

# Iterate over the rows with missing values in water quality columns
for index, row in df[df[water_quality_cols].isnull().any(axis=1)].iterrows():
    # Extract the month, year, and monitoring station of the row with missing values
    month, year, station = row['Month'], row['Year'], row['Monitoring Stations']
    
    # Filter data for the same monitoring station and month in different years
    similar_months = df[(df['Month'] == month) & (df['Year'] != year) & 
                        (df['Monitoring Stations'] == station) & 
                        (df['Phytoplankton (cells/ml)'].notnull())]
    
    # Compute similarity based on weather columns and phytoplankton
    min_distance = float('inf')
    best_match = None
    
    for i, other_row in similar_months.iterrows():
        weather_distance = euclidean(row[weather_cols], other_row[weather_cols])
        if phytoplankton_distance(row, other_row) and weather_distance < min_distance:
            min_distance = weather_distance
            best_match = other_row
    
    # If a best match is found, fill in the missing values
    if best_match is not None:
        for col in water_quality_cols:
            if pd.isnull(row[col]):
                df.at[index, col] = best_match[col]

# Save the modified DataFrame
output_path = 'Filled_Merged_Phytoplankton.csv'
df.to_csv(output_path, index=False)





In [49]:
import pandas as pd
from sklearn.impute import KNNImputer

# Load your data into the DataFrame 'merged_df'
merged_df = pd.read_csv('New_Merged.csv', encoding='utf-8')  # Replace with the correct file path and encoding

#threshold = len(merged_df.columns) - 6
#merged_df = merged_df.dropna(thresh=threshold)

# Specify the columns to impute
columns_to_impute = [
    'pH (units)', 
    'Ammonia (mg/L)', 
    'Nitrate (mg/L)', 
    'Inorganic Phosphate (mg/L)', 
    'BOD (mg/l)', 
    'Dissolved Oxygen (mg/l)', 
    'Total coliforms (MPN/100ml)'
]

# Extract the columns to impute
data_to_impute = merged_df[columns_to_impute]

# Create the imputer
imputer = KNNImputer(n_neighbors=5)

# Apply the imputer
data_imputed = imputer.fit_transform(data_to_impute)

# Convert the imputed data back to a DataFrame
data_imputed_df = pd.DataFrame(data_imputed, columns=columns_to_impute)

# Replace the original columns in the DataFrame with the imputed data
merged_df[columns_to_impute] = data_imputed_df

# Save the DataFrame with imputed values to a new CSV file
merged_df.to_csv('New_Merged_Imputed.csv', index=False)  # Replace with your desired file path


In [17]:
import pandas as pd

# Load your data into the DataFrame 'merged_df'
merged_df = pd.read_csv('New_Merged.csv', encoding='utf-8')  # Replace with the correct file path and encoding

# Print the column names
print(merged_df.columns)


Index(['Monitoring Stations', 'Month', 'Year', 'pH (units)', 'Ammonia (mg/L)',
       'Nitrate (mg/L)', 'Inorganic Phosphate (mg/L)', 'BOD (mg/l)',
       'Dissolved Oxygen (mg/l)', 'Total coliforms (MPN/100ml)',
       'Phytoplankton (cells/ml)', 'Wind', 'Condition', 'Day', 'Time',
       'Temperature', 'Dew Point', 'Humidity', 'Wind Speed', 'Wind Gust',
       'Pressure', 'Precip.', 'Date'],
      dtype='object')


In [ ]:
import pandas as pd
import pickle
from sklearn.preprocessing import StandardScaler

# Load the full dataset
df = pd.read_csv('2023_merged.csv')

# Filter the dataset to include only rows from the year 2023 for prediction
df_2023 = df[df['Year'] == 2023]

# Select the features used for prediction
X_2023 = df_2023[['Temperature', 'Humidity', 'Wind', 'Wind Speed', 'Ammonia (mg/L)', 'Inorganic Phosphate (mg/L)', 'BOD (mg/l)']]

# Load the saved model
with open('xgb_model.pkl', 'rb') as file:
    loaded_xgb_model = pickle.load(file)

# Preprocess the data if necessary (e.g., scaling)

# Load the scaler
with open('xgb_scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

scaler = StandardScaler()
X_2023_scaled = scaler.fit_transform(X_2023)

# Predict phytoplankton counts for 2023 data
phytoplankton_predictions = loaded_xgb_model.predict(X_2023_scaled)

# Add predictions to the 2023 dataframe
df_2023['Predicted Phytoplankton (cells/ml)'] = phytoplankton_predictions

# Merge the 2023 predictions back into the original dataset
df_final = pd.merge(df, df_2023[['Year', 'Month', 'Day', 'Predicted Phytoplankton (cells/ml)']], 
                    on=['Year', 'Month', 'Day'], how='left')

# Save the updated dataset with predictions to a new CSV file
df_final.to_csv('final_dataset_with_predictions.csv', index=False)

print("Final dataset saved to 'final_dataset_with_predictions.csv'")


In [54]:
import pandas as pd
import pickle
from sklearn.preprocessing import StandardScaler

# Load the full dataset
df = pd.read_csv('Complete.csv')

# Filter the dataset to include only rows from the year 2023 for prediction
df_2023 = df[df['Year'] == 2023]

# Select the features used for prediction
X_2023 = df_2023[['Temperature', 'Humidity', 'Wind', 'Wind Speed', 'Ammonia (mg/L)', 'Inorganic Phosphate (mg/L)', 'BOD (mg/l)']]

# Load the saved model
with open('xgb_model.pkl', 'rb') as file:
    loaded_xgb_model = pickle.load(file)

# Load the scaler
with open('xgb_scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

# Scale the data using the loaded scaler
X_2023_scaled = scaler.transform(X_2023)

# Predict phytoplankton counts for 2023 data
phytoplankton_predictions = loaded_xgb_model.predict(X_2023_scaled)

# Round off the predictions to the nearest whole number
phytoplankton_predictions = phytoplankton_predictions.round()

# Update the 'Phytoplankton (cells/ml)' column with rounded predictions for 2023
df.loc[df['Year'] == 2023, 'Phytoplankton (cells/ml)'] = phytoplankton_predictions

# Save the updated dataset to a new CSV file
df.to_csv('updated_dataset_with_predictions.csv', index=False)

print("Dataset with predictions updated and saved to 'updated_dataset_with_predictions.csv'")


Dataset with predictions updated and saved to 'updated_dataset_with_predictions.csv'


In [ ]:
# Sample input data (replace with realistic values)
sample_data = pd.DataFrame({
    'Temperature': [28.29166667],  # Example temperature
    'Humidity': [65.91666667],     # Example humidity
    'Wind': [6],            # Example wind direction (mapped value)
    'Wind Speed': [10.58333333],   # Example wind speed
    'Condition': [1],       # Example weather condition (mapped value)
    'pH (units)': [8],    # Example pH level
    'Ammonia (mg/L)': [0.057], # Example ammonia level
    'Nitrate (mg/L)': [ 0.357], # Example nitrate level
    'Inorganic Phosphate (mg/L)': [0.059], # Example phosphate level
    'BOD (mg/l)': [2],    # Example BOD level
    'Dissolved Oxygen (mg/l)': [7.9], # Example dissolved oxygen level
    'Total coliforms (MPN/100ml)': [410] # Example total coliforms
})

# Time Series Data Cleaning

In [25]:
import pandas as pd
from sklearn.impute import KNNImputer

df = pd.read_csv("updated_dataset_with_predictions.csv")

keep = ['Monitoring Stations', 'Month', 'Year', 'Date', 'pH (units)', 'Ammonia (mg/L)', 'Inorganic Phosphate (mg/L)', 'BOD (mg/l)', 'Total coliforms (MPN/100ml)', 'Phytoplankton (cells/ml)']

df_dropped = df[keep]

print(df_dropped.columns)

missing = df_dropped.isnull().sum()

print(missing)
df_dropped.to_csv('dropped_columns.csv', index=False)

drop = pd.read_csv('dropped_columns.csv')


impute_columns = ['pH (units)', 'Ammonia (mg/L)', 'Inorganic Phosphate (mg/L)', 'BOD (mg/l)', 'Total coliforms (MPN/100ml)']
imputer = KNNImputer(n_neighbors=5)
drop[impute_columns] = imputer.fit_transform(drop[impute_columns])

drop = drop.dropna(subset=['Monitoring Stations'])


missing2 = drop.isnull().sum()

print(missing2)

drop.to_csv('Knn_time.csv', index=False)


Index(['Monitoring Stations', 'Month', 'Year', 'Date', 'pH (units)',
       'Ammonia (mg/L)', 'Inorganic Phosphate (mg/L)', 'BOD (mg/l)',
       'Total coliforms (MPN/100ml)', 'Phytoplankton (cells/ml)'],
      dtype='object')
Monitoring Stations             12
Month                           12
Year                            12
Date                            12
pH (units)                     237
Ammonia (mg/L)                 425
Inorganic Phosphate (mg/L)     399
BOD (mg/l)                     206
Total coliforms (MPN/100ml)    229
Phytoplankton (cells/ml)        58
dtype: int64
Monitoring Stations             0
Month                           0
Year                            0
Date                            0
pH (units)                      0
Ammonia (mg/L)                  0
Inorganic Phosphate (mg/L)      0
BOD (mg/l)                      0
Total coliforms (MPN/100ml)     0
Phytoplankton (cells/ml)       46
dtype: int64


In [14]:
import pandas as pd
from sklearn.impute import KNNImputer

df = pd.read_csv("updated_dataset_with_predictions.csv")

keep = ['Monitoring Stations', 'Month', 'Year', 'Date', 'pH (units)', 'Ammonia (mg/L)', 'Inorganic Phosphate (mg/L)', 'BOD (mg/l)', 'Total coliforms (MPN/100ml)', 'Phytoplankton (cells/ml)']

df_dropped = df[keep]

print(df_dropped.columns)

missing = df_dropped.isnull().sum()

print(missing)
df_dropped.to_csv('dropped_columns.csv', index=False)

drop = pd.read_csv('dropped_columns.csv')


# List of columns to fill based on their correlation with Phytoplankton (cells/ml)
columns_to_fill = ['pH (units)', 'Ammonia (mg/L)', 'Inorganic Phosphate (mg/L)', 'BOD (mg/l)', 'Total coliforms (MPN/100ml)']

# Define the predictors to be used
predictors = ['Phytoplankton (cells/ml)']

# Create a subset of the DataFrame including the predictors and the columns to fill
df_subset = drop[predictors + columns_to_fill]

# Initialize the MICE (Iterative Imputer)
mice_imputer = IterativeImputer(max_iter=10, random_state=0)

# Apply MICE imputation to the relevant subset of the DataFrame
df_imputed = pd.DataFrame(mice_imputer.fit_transform(df_subset), columns=df_subset.columns)

# Replace the original columns with the imputed ones
drop[columns_to_fill] = df_imputed[columns_to_fill]

# Check if missing values are filled
print(drop[columns_to_fill].isnull().sum())

# Save the updated dataset with imputed values to a new CSV file
drop.to_csv('MICE_Time.csv', index=False)


Index(['Monitoring Stations', 'Month', 'Year', 'Date', 'pH (units)',
       'Ammonia (mg/L)', 'Inorganic Phosphate (mg/L)', 'BOD (mg/l)',
       'Total coliforms (MPN/100ml)', 'Phytoplankton (cells/ml)'],
      dtype='object')
Monitoring Stations             12
Month                           12
Year                            12
Date                            12
pH (units)                     237
Ammonia (mg/L)                 425
Inorganic Phosphate (mg/L)     399
BOD (mg/l)                     206
Total coliforms (MPN/100ml)    229
Phytoplankton (cells/ml)        58
dtype: int64
pH (units)                     0
Ammonia (mg/L)                 0
Inorganic Phosphate (mg/L)     0
BOD (mg/l)                     0
Total coliforms (MPN/100ml)    0
dtype: int64
